In [1]:
import os
from tqdm import tqdm

import pandas as pd
import numpy as np

import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

In [148]:
X_train = torch.randn(100,2)
Y_train = torch.tensor([[(a.item()+b.item())/2] for a,b in X_train])
X_test = torch.randn(100,2)
Y_test = torch.tensor([[(a.item()+b.item())/2] for a,b in X_test])
print(X_train[:5])
print(Y_train[:5])

tensor([[ 0.3335, -0.7630],
        [ 1.0933, -0.9819],
        [-0.5483, -0.7565],
        [-0.5008,  0.1574],
        [ 0.1913, -0.7973]])
tensor([[-0.2147],
        [ 0.0557],
        [-0.6524],
        [-0.1717],
        [-0.3030]])


In [145]:
class myDataset(Dataset):
  def __init__(self,X,Y):
    self.X=X
    self.Y=Y
  def __getitem__(self,index):
    return self.X[index], self.Y[index]
  def __len__(self):
    return len(self.X)

In [146]:
class myModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(2,10)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(10,1)
  def forward(self,x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

In [147]:
EPOCHS = 1000
BATCH_SIZE = 32

In [158]:
def train(model, data_loader, loss_fn, optimizer):
  model.train()
  for epoch in range(EPOCHS):
    running_loss = 0
    for x,y in data_loader:
      optimizer.zero_grad()
      y_pred = model(x)
      loss = loss_fn(y_pred,y)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
    running_loss /= len(data_loader)
    if epoch % 100 == 0:
      print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss:.4f}')

def test(model, data_loader, loss_fn):
  running_loss = 0
  model.eval()
  gt = None
  pred = None
  with torch.no_grad():
    for x,y in data_loader:
      y_pred = model(x)
      gt = y if gt is None else torch.cat((gt,y))
      pred = y_pred if pred is None else torch.cat((pred,y_pred))
      loss = loss_fn(y_pred,y)
      running_loss += loss.item()
  return gt, pred, running_loss/len(data_loader)

In [160]:
datasets = {
  "train" : myDataset(X_train, Y_train),
  "test" : myDataset(X_test, Y_test)
}
dataloaders = {
  "train" : DataLoader(datasets["train"],batch_size=BATCH_SIZE,shuffle=True),
  "test" : DataLoader(datasets["test"],batch_size=BATCH_SIZE,shuffle=False)
}

model = myModel()
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [190]:
train(model,dataloaders["train"],loss_fn,optimizer)

Epoch [1/1000], Loss: 0.0000
Epoch [101/1000], Loss: 0.0000
Epoch [201/1000], Loss: 0.0000
Epoch [301/1000], Loss: 0.0000
Epoch [401/1000], Loss: 0.0000
Epoch [501/1000], Loss: 0.0000
Epoch [601/1000], Loss: 0.0000
Epoch [701/1000], Loss: 0.0000
Epoch [801/1000], Loss: 0.0000
Epoch [901/1000], Loss: 0.0000


In [191]:
gt, pred, loss = test(model,dataloaders["test"],loss_fn)
print("loss:",loss)
print(gt[:5])
print(pred[:5])

loss: 8.896838210148417e-06
tensor([[ 0.1892],
        [-0.1273],
        [-0.0722],
        [-0.2635],
        [ 0.9604]])
tensor([[ 0.1886],
        [-0.1269],
        [-0.0714],
        [-0.2618],
        [ 0.9585]])


In [192]:
X_test = torch.randn(5,2)
Y_test = torch.tensor([[(a.item()+b.item())/2] for a,b in X_test])
Y_test_pred = model(X_test)
for i in range(5):
  print(Y_test[i].item(),",",Y_test_pred[i].item())

1.8639471530914307 , 1.8632662296295166
-0.9432952404022217 , -0.9463598728179932
-0.6888329982757568 , -0.6884360313415527
0.2806812822818756 , 0.2807430922985077
1.6530423164367676 , 1.651261806488037
